# Real "sera" data example
In this notebook we fit real data from a deep mutational scanning experiment that used a cocktail containing two monoclonal antibodies. Thus, this can be interpreted as a polyclonal serum with a pre-defined composition.

The monoclonal antibodies in question are 1C04 and 5G04, and the deep mutational scanning is against the Influenza A/H3/HongKong/45/2019 Hemagglutinin, using a library designed to contain only mutations that are thought to be well tolerated.

First, import the Python modules:

In [1]:
import time

import altair as alt

import polyclonal

import pandas as pd

## Get the data to fit
Now we read the deep mutational scanning measurements, which quantify the "probability of escape" (fraction not neutralized) for each variant. For a description of key columns, see [here](https://jbloomlab.github.io/polyclonal/real_LyCoV1404.html).

In [2]:
prob_escape = pd.read_csv(
    "libA_220810_1_1C04-5G04_1_prob_escape.csv", keep_default_na=False, na_values="nan"
).query(
    "`no-antibody_count` >= no_antibody_count_threshold"
)  # filter for those with sufficient no-antibody counts
assert prob_escape.notnull().all().all()
prob_escape.head()

,library,antibody_sample,no-antibody_sample,aa_substitutions_sequential,n_aa_substitutions,barcode,prob_escape,prob_escape_uncensored,antibody_count,no-antibody_count,antibody_neut_standard_count,no-antibody_neut_standard_count,total_no_antibody_count,no_antibody_count_threshold,aa_substitutions_reference,antibody,antibody_concentration
0,libA,220810_1_antibody_1C04-5G04_3.65_1,220810_1_no-antibody_control_1,K297I,1,ATAACACAAAAAAGTA,0.0017,0.0017,78972,246344,8599550,44895,10428350,15,K278I,1C04-5G04,3.65
1,libA,220810_1_antibody_1C04-5G04_3.65_1,220810_1_no-antibody_control_1,R111S V366M R402S,3,TATCTACCTAACGAAA,0.0047,0.0047,70662,78014,8599550,44895,10428350,15,R92S V347M R383S,1C04-5G04,3.65
2,libA,220810_1_antibody_1C04-5G04_3.65_1,220810_1_no-antibody_control_1,A125M P246H I393A G398Q F411Y,5,CTTTCAATTATGAGAC,0.0370,0.0370,57908,8163,8599550,44895,10428350,15,A106M P227H I374A G379Q F392Y,1C04-5G04,3.65
3,libA,220810_1_antibody_1C04-5G04_3.65_1,220810_1_no-antibody_control_1,Y113M S143N S164N I307M I393Y E468Q,6,TGTATTAGCATTTTGA,0.0074,0.0074,37740,26593,8599550,44895,10428350,15,Y94M S124N S145N I288M I374Y E449Q,1C04-5G04,3.65
4,libA,220810_1_antibody_1C04-5G04_3.65_1,220810_1_no-antibody_control_1,G237H P246H V366M,3,CCAAGGAGCACGAAAA,0.0218,0.0218,26699,6407,8599550,44895,10428350,15,G218H P227H V347M,1C04-5G04,3.65


Display the number of variants per concentration.

In [3]:
display(
    prob_escape.groupby("antibody_concentration").aggregate(
        n_variants=pd.NamedAgg("barcode", "nunique")
    )
)

,n_variants
antibody_concentration,
0.114,31001
0.228,31001
0.456,31001
0.913,31001
1.825,31001
3.650,31001


Plot mean probability of escape across all variants with the indicated number of mutations. Note that this plot weights each variant the same in the means regardless of how many barcode counts it has. We plot means for both censored (set to between 0 and 1) and uncensored probabilities of escape. Also, note it uses a symlog scale for the y-axis. Mouseover points for values:



In [4]:
# NBVAL_IGNORE_OUTPUT
max_aa_subs = 4  # group if >= this many substitutions

mean_prob_escape = (
    prob_escape.assign(
        n_subs=lambda x: (
            x["aa_substitutions_sequential"]
            .str.split()
            .map(len)
            .clip(upper=max_aa_subs)
            .map(lambda n: str(n) if n < max_aa_subs else f">{max_aa_subs - 1}")
        )
    )
    .groupby(["antibody_concentration", "n_subs"], as_index=False)
    .aggregate({"prob_escape": "mean", "prob_escape_uncensored": "mean"})
    .rename(
        columns={
            "prob_escape": "censored to [0, 1]",
            "prob_escape_uncensored": "not censored",
        }
    )
    .melt(
        id_vars=["antibody_concentration", "n_subs"],
        var_name="censored",
        value_name="probability escape",
    )
)

mean_prob_escape_chart = (
    alt.Chart(mean_prob_escape)
    .encode(
        x=alt.X("antibody_concentration"),
        y=alt.Y(
            "probability escape",
            scale=alt.Scale(type="symlog", constant=0.05),
        ),
        column=alt.Column("censored", title=None),
        color=alt.Color("n_subs", title="n substitutions"),
        tooltip=[
            alt.Tooltip(c, format=".3g") if mean_prob_escape[c].dtype == float else c
            for c in mean_prob_escape.columns
        ],
    )
    .mark_line(point=True, size=0.5)
    .properties(width=200, height=125)
    .configure_axis(grid=False)
)

mean_prob_escape_chart

/home/tyu2/.conda/envs/polyclonal_escape/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

By default, `polyclonal` fits a model without regularizing the similarity of escape values across epitopes. We can try this first by fitting two epitopes, filtering out data measured at concentrations < 1.0 as these are too noisy.

In [5]:
# NBVAL_IGNORE_OUTPUT
model = polyclonal.Polyclonal(
    n_epitopes=2,
    data_to_fit=prob_escape.rename(
        columns={
            "antibody_concentration": "concentration",
            "aa_substitutions_sequential": "aa_substitutions",
        }
    ).query("concentration > 1.0"),
    alphabet=polyclonal.AAS_WITHSTOP_WITHGAP,
)

# fit model
opt_res = model.fit(
    logfreq=200,
    reg_escape_weight=0.1,
)
model.mut_escape_plot(category_colors={"1": "#0072B2", "2": "#CC79A7"})

# First fitting site-level model.
# Starting optimization of 1026 parameters at Wed Oct 12 10:24:11 2022.
         step     time_sec         loss     fit_loss   reg_escape   reg_spreadreg_similarity reg_activity
            0     0.035846       264.24       263.33            0            0            0      0.90499
           56       2.0037       125.73       121.52       2.9785            0            0       1.2287
# Successfully finished at Wed Oct 12 10:24:13 2022.
# Starting optimization of 6736 parameters at Wed Oct 12 10:24:13 2022.
         step     time_sec         loss     fit_loss   reg_escape   reg_spreadreg_similarity reg_activity
            0     0.038325        186.2       157.37       27.606   8.4381e-32            0       1.2287
           70       2.9669       164.39       152.36       10.271       0.3391            0       1.4259
# Successfully finished at Wed Oct 12 10:24:16 2022.


/home/tyu2/.conda/envs/polyclonal_escape/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

Note that `polyclonal` fits the same escape values for both epitopes and fails to deconvolve the polyclonal serum. We can resolve this by regularizing the similarity of escape values across epitopes.

In [17]:
# NBVAL_IGNORE_OUTPUT
model = polyclonal.Polyclonal(
    n_epitopes=2,
    data_to_fit=prob_escape.rename(
        columns={
            "antibody_concentration": "concentration",
            "aa_substitutions_sequential": "aa_substitutions",
        }
    ).query("concentration > 1.0"),
    alphabet=polyclonal.AAS_WITHSTOP_WITHGAP,
)

# fit model
opt_res = model.fit(
    logfreq=200,
    reg_escape_weight=0.1,
    reg_similarity_weight=0.24,  # regularize epitope similarity
    reg_similarity_epsilon=1e-3,
)
model.mut_escape_plot(category_colors={"1": "#0072B2", "2": "#CC79A7"})

# First fitting site-level model.
# Starting optimization of 1026 parameters at Wed Oct 12 10:30:41 2022.
         step     time_sec         loss     fit_loss   reg_escape   reg_spreadreg_similarity reg_activity
            0     0.032947       264.36       263.33            0            0      0.12288      0.90499
          182       6.8512       126.53       121.86       2.8474            0      0.58194        1.238
# Successfully finished at Wed Oct 12 10:30:48 2022.
# Starting optimization of 6736 parameters at Wed Oct 12 10:30:48 2022.
         step     time_sec         loss     fit_loss   reg_escape   reg_spreadreg_similarity reg_activity
            0     0.036515       189.99       157.66       27.652   2.3161e-31       3.4453        1.238
          116       4.8793       166.84       154.89       8.6953      0.35857       1.4506       1.4426
# Successfully finished at Wed Oct 12 10:30:53 2022.


/home/tyu2/.conda/envs/polyclonal_escape/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)